In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
# ws.get_details()



Ready to use Azure ML 1.42.0 to work with tcp0044aedcmluc001dtmlws


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tcp0044uc001cc03"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [3]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'ASPR_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

ASPR_pipeline


In [4]:
%%writefile $experiment_folder/python_env2.yml

name: python_env2

dependencies:
- python
- anaconda
- pip
- pip:
      - azureml.sdk
      - sklearn
      - statsmodels
      - pyarrow
      - numpy
      - pandas
      - seaborn
      - joblib
      - shap
      - graphviz 
      - tensorflow
      - plotly
      - pmdarima
      - emmv
           

Overwriting ASPR_pipeline/python_env2.yml


In [5]:
%%writefile $experiment_folder/aspr_script.py
#!/usr/bin/env python
# coding: utf-8

from __future__ import print_function
import pandas as pd
import numpy as np
import sklearn
from sklearn.covariance import EllipticEnvelope
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
#import plotly.graph_objects as go
#%matplotlib inline
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
#from emmv import emmv_scores

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import os
from datetime import date
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARMA
#from statsmodelstsaarima_model
from joblib import Parallel, delayed
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.api import VAR
import joblib
import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import pandas as pd
import numpy as np
from sklearn.covariance import EllipticEnvelope
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.covariance import EllipticEnvelope
#%matplotlib inline
#from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
import shap 
#import graphviz
sns.set_style('darkgrid')
from sklearn.decomposition import PCA
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import chart_studio.plotly as py
import matplotlib.pyplot as plt
from matplotlib import pyplot
import plotly.graph_objs as go
from sklearn.svm import OneClassSVM
from numpy import where
init_notebook_mode(connected=True)
import numpy as np
import pandas as pd
#import theano
#import theano.tensor as T

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
#from emmv import emmv_scores
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, InputLayer
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
#from keras.utils import np_utils
from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.layers.embeddings import Embedding
#from tensorflow.keras.layers.recurrent import SimpleRNN
#from sklearn.cross_validation import train_test_split
#from tensorflow.keras.layers.core import Activation, RepeatVector
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from tensorflow.keras.layers import TimeDistributed
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
from azureml.core.run import Run

# subscription_id = 'c4f0bb45-07a9-489a-8f6a-6fbc64f25b46'
# resource_group = 'tcp-azu0044-ae-rg-dcml-uc001-dt'
# workspace_name = 'tcp0044aedcmluc001dtmlws'

# workspace = Workspace(subscription_id, resource_group, workspace_name)

workspace = Run.get_context().experiment.workspace
run = Run.get_context()
ws = run.experiment.workspace

ASPR_data = Dataset.get_by_name(workspace, name='Cycletime_2022')
ASPR_data = ASPR_data.to_pandas_dataframe()

ASPR_data_new = ASPR_data.loc[: ,['OrderN','CycleTim','BusinessUnit','OrderTypeKey','TechType','InstallationTypeKey','AgeRangeKey','State','IndustryPartnerName','CustomerTypeKey','OrderChannel','NBNServiceClass','Diff_SIK_HardwareTCD','Stage1_Cycletime','Stage2_Cycletime','Stage3_Cycletime','Stage4_Cycletime','FieldActivityStateKey','PartialCompleteCntKey','FieldActivityRescheduleStateKey','RescheduleDriver','RescheduleCntKey','ItamNumber','SubmitToResolve']]

ASPR_data_new['State'] = ASPR_data_new['State'].fillna('NA')
ASPR_data_new['OrderChannel'] = ASPR_data_new['OrderChannel'].fillna('NA')
ASPR_data_new['NBNServiceClass'] = ASPR_data_new['NBNServiceClass'].fillna('NA')
ASPR_data_new['Diff_SIK_HardwareTCD'] = ASPR_data_new['Diff_SIK_HardwareTCD'].fillna(0)
ASPR_data_new['Stage1_Cycletime'] = ASPR_data_new['Stage1_Cycletime'].fillna(0)
ASPR_data_new['Stage2_Cycletime'] = ASPR_data_new['Stage2_Cycletime'].fillna(0)
ASPR_data_new['Stage3_Cycletime'] = ASPR_data_new['Stage3_Cycletime'].fillna(0)
ASPR_data_new['Stage4_Cycletime'] = ASPR_data_new['Stage4_Cycletime'].fillna(0)
ASPR_data_new['FieldActivityStateKey'] = ASPR_data_new['FieldActivityStateKey'].fillna('NA')
ASPR_data_new['PartialCompleteCntKey'] = ASPR_data_new['PartialCompleteCntKey'].fillna(0)
ASPR_data_new['FieldActivityRescheduleStateKey'] = ASPR_data_new['FieldActivityRescheduleStateKey'].fillna('NA')
ASPR_data_new['RescheduleDriver'] = ASPR_data_new['RescheduleDriver'].fillna('NA')
ASPR_data_new['RescheduleCntKey'] = ASPR_data_new['RescheduleCntKey'].fillna(0)
ASPR_data_new['ItamNumber'] = ASPR_data_new['ItamNumber'].fillna(0)
ASPR_data_new['SubmitToResolve'] = ASPR_data_new['SubmitToResolve'].fillna(0)

def condition(x):
    if x>0:
        return "Customer delay"
    elif x<0:
        return "Telstra delay"
    else:
        return "NA"

ASPR_data_new['Diff_SIK_HardwareTCD'] = ASPR_data_new['Diff_SIK_HardwareTCD'].apply(condition)

ASPR_data_new['BU_Ordertype'] = ASPR_data_new['BusinessUnit']+ '-' +ASPR_data_new['OrderTypeKey']

ASPR_data_new=ASPR_data_new.drop(['BusinessUnit','OrderTypeKey'], axis=1)

ASPR_data_new1 = pd.get_dummies(ASPR_data_new,columns = ['TechType', 'InstallationTypeKey', 'AgeRangeKey', 'State','IndustryPartnerName', 'CustomerTypeKey', 'OrderChannel','NBNServiceClass','Diff_SIK_HardwareTCD', 'FieldActivityStateKey', 'FieldActivityRescheduleStateKey', 'RescheduleDriver'])

def drop_allzero_columns(df):
    for i in df.columns:
        if ((df[i] == 0.0).all() == True ):
            df.drop([i], inplace=True, axis=1)
    return df

df_final = drop_allzero_columns(ASPR_data_new1)


# # Testing Causation using Granger’s Causality Test

from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df


df_final['CycleTim'] = pd.to_numeric(df_final['CycleTim'],errors='coerce').fillna(0)

df_final = df_final.rename(columns = {'CycleTim':'Target_var','BU_Ordertype':'Category'}, inplace = False)

df_final_numericcolumns = df_final.select_dtypes(include =['float64','int64','uint8'])

correlation= df_final_numericcolumns.corr(method='pearson')

sorted_mat = correlation.unstack().sort_values()

batch_folder = 'Output'
os.makedirs(batch_folder, exist_ok=True)

# Extract all the files to csv
local_path1 = 'Output/asprcorrelation.csv'
correlation.to_csv(local_path1)

variables1 = df_final_numericcolumns.columns


# X causing Y. If less than significance level, its good. for eg less than 0.05

#df_final_causality = grangers_causation_matrix(df_final, variables = variables1)

# Extract all the files to csv
#local_path2 = 'Output/df_final_causality.csv'
#df_final_causality.to_csv(local_path2)

#df_causality = df_final_causality.loc[df_final_causality.index == 'Target_var_y']

#def drop_lesscausality(df):
    #droppedcols=[]
#    for i in df.columns:
#        if (df[i][0] > 0.05):
#            df.drop([i], inplace=True, axis=1)
#            #droppedcols = droppedcols+i
#    return df

#df1 = drop_lesscausality(df_causality)

#k = []
#def drop_causality(df):
#    for i in df.columns:
#        j=i[:-2]
#        k.append(j)
#    return k


#list1 = drop_causality(df1)

#df3 = df_final[list1]
df3 = df_final
#df3['Week_Ending'] = df_final['Week_Ending']
#df3['Category'] = df_final['Category'] 
#df3['Target_var'] = df_final['Target_var']

#dropped = list(set(variables1)-set(list1))

cor = df3.corr()


#Correlation with output variable
cor_target = abs(cor["Target_var"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.20]

#Using lasso to select important features

X = df3.drop(["Target_var",'OrderN','Category'],1)   #Feature Matrix
y = df3["Target_var"]

from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

reg = LassoCV()
reg.fit(X, y)
#print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
#print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

#print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

#imp_coef = coef.sort_values()
#import matplotlib
#matplotlib.rcParams['figure.figsize'] = (8.0, 20.0)
#imp_coef.plot(kind = "barh")
#plt.title("Feature importance using Lasso Model")


# ### common functions used in the below code


#A helper function to find percentage change,classify anomaly based on severity.
#The predict function classifies the data as anomalies based on the results from decision function on crossing a threshold.
#Say if the business needs to find the next level of anomalies which might have an impact, this could be used to identify those 
#points.The top 30 quantiles are identified anomalies(high severity), based on decision function here we identify the 30–60 
#quantile points and classify them as low severity anomalies.

def classify_anomalies(df,metric_name):
    df['metric_name']=metric_name
    df = df.sort_values(by='OrderN', ascending=False)
    #Shift actuals by one timestamp to find the percentage chage between current and previous data point
    df['shift'] = df['actuals'].shift(-1)
    df['percentage_change'] = ((df['actuals'] - df['shift']) / df['actuals']) * 100
    return df

listofcategory=df3['Category'].unique()

#df3 = df3.loc[df3['Week_Ending'] < '2022-02-01']


# ### Using the isolation forest algorithm for anomaly detection, explaining the anomalies using percentage change of feature values from previous week ending and then SHAP to get the important features

anomaly_iforest_IQR = pd.DataFrame()
featureimportance_global = pd.DataFrame()
featureimportance_local = pd.DataFrame()
Activity_percentage_change = pd.DataFrame()

for i in listofcategory:
    #print(i)
    
    data=df3[df3['Category']== i]
    
    data = data.drop(['Category'],axis=1)
    data = data.groupby(['OrderN']).sum()
    
    data4 = data.copy()
    data4['change percentage'] = data4['Target_var'].pct_change()
    data4['change percentage'] = abs(data4['change percentage'])
    data4['change percentage'] = data4['change percentage'].replace(np.inf, max(data4['change percentage']))
    data4 = data4.fillna(0)
    avgprcntgechnge = data4['change percentage'].mean()
    #print(avgprcntgechnge)
    n=50
    data5 = data4.iloc[-n:]

    count = (data5[data5['change percentage'] > avgprcntgechnge]['change percentage'].count())
    contaminationfactor = count/n
    #print(count)
    #print(contaminationfactor)

    factor = max(0.1,contaminationfactor)
    #print(factor)
    
    model=IsolationForest(n_estimators=10, max_samples='auto', contamination=factor,                             max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
    model.fit(data[data.columns])
    data1=data.copy()
    data1['scores']=model.decision_function(data[data.columns])
    data1['anomaly_iforest']=model.predict(data[data.columns])
    data1.loc[data1['anomaly_iforest'] == 1,'anomaly_iforest'] = 0
    data1.loc[data1['anomaly_iforest'] == -1,'anomaly_iforest'] = 1

    #fig = px.line(data1,x=data1.index,y="Target_var",template = 'plotly_dark')
    # create list of outlier_dates
    #outlier_dates = data1[data1['anomaly_iforest'] == 1].index
    # obtain y value of anomalies to plot
    #y_values = [data1.loc[i]['Target_var'] for i in outlier_dates]
    #fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
    #                name = i + "_" + "Anomaly", 
    #                marker=dict(color='red',size=10)))
    #plt.legend(loc="upper center")
    #fig.show()
    
    high_thresh=(1.0+factor)
    low_thresh=(1.0-factor)
    # generate the labels using 4 week rolling average technique and IQR technique for evaluation
    #data1['4W rolling average'] = abs(data1.Target_var).rolling(4).mean()
    #data1['lag1'] = data1['Target_var'].shift(1)
    #data1 = data1.reset_index()
    #for p in range(len(data1)):
    #    if data1.loc[p,'4W rolling average'] != "nan":
    #        if (abs(data1.loc[p,'Target_var']) > (data1.loc[p,'4W rolling average']*high_thresh)) or (abs(data1.loc[p,'Target_var']) < (data1.loc[p,'4W rolling average']*low_thresh)):
    #            data1.loc[p,'Anomaly_4wra'] = 1
    #        else:
    #            data1.loc[p,'Anomaly_4wra'] = 0      
    #data1= data1.set_index('OrderN') 
    #data1= data1.fillna(0)

    def iqr_bounds(scores):
            q1 = scores.quantile((factor/2))
            q3 = scores.quantile(1.0-(factor/2))
            #iqr = q3 - q1
            lower_bound=(q1)
            upper_bound=(q3)
            #print("Lower bound:{} \nUpper bound:{} \nq1:{} \nq3:{}".format(lower_bound,upper_bound,q1,q3))
            return lower_bound,upper_bound

    lower_bound,upper_bound=iqr_bounds(data1['Target_var'])

    data1['Anomaly_Label_IQR']=0
    data1['Anomaly_Label_IQR']=(data1['Target_var'] < lower_bound) |(data1['Target_var'] > upper_bound)
    data1['Anomaly_Label_IQR']=data1['Anomaly_Label_IQR'].astype(int)  
    
    
    # get the global feature importances at segment-product level using SHAP
    exp = shap.TreeExplainer(model) #Explainer
    shap_values = exp.shap_values(data)  #Calculate SHAP values
    #shap.summary_plot(shap_values, data, plot_type="bar", title=i+ "-" + "Global importance")
    data2 = pd.DataFrame()
    for j in range(0, len(data)):
        data2=data2.append(dict(zip(data.columns, shap_values[j])), ignore_index=True)
    
    # get the local feature importances at each index level for every segment-product combination using SHAP
    explainer = shap.TreeExplainer(model, data=data[data.columns])
    shapvs = explainer.shap_values(data[data.columns], check_additivity=True) # check should be default
    data3=pd.DataFrame()
    for l in range(0,len(data)):
        data3 = data3.append(dict(zip(data.columns, shapvs[l])), ignore_index=True)
    
    
    
    # visually see if the anomalies identified seem reasonable by feeding data to PCA reduced to 2 dimensions
    data_2=data1.copy()
    data_2=data_2.reset_index()
    outliers=data_2.loc[data_2['anomaly_iforest']== 1]
    outlier_index=list(outliers.index)
    
    #pca = PCA(2)
    #pca.fit(data1[data.columns])
    #res=pd.DataFrame(pca.transform(data1[data.columns]))
    #Z = np.array(res)
    #plt.title("IsolationForest")
    #plt.contourf( Z, cmap=plt.cm.Blues_r)
    #b1 = plt.scatter(res[0], res[1], c='green',
    #                 s=10,label="normal points")
    #b1 =plt.scatter(res.iloc[outlier_index,0],res.iloc[outlier_index,1], c='green',s=10,  edgecolor="red")
    #plt.legend(loc="upper center")
    #plt.show()
    
    # Identify anomalies for individual metrics and plot the results.
    # X axis — date
    # Y axis — Actual values and anomaly points.
    # Actual values of metrics are indicated in the blue line and anomaly points are highlighted as red points.
    # In the table, the background red indicates high anomalies and yellow indicates low anomalies.
    #features_perchange=pd.DataFrame()
    #for m in range(1,len(data_2.columns)-1):
    #    test_df=pd.DataFrame()
    #    test_df['OrderN']=data_2['OrderN']
    #    test_df['actuals']=data_2.iloc[:,m:m+1]
    #    test_df=classify_anomalies(test_df,data_2.columns[m])
    #    features_perchange = features_perchange.append(test_df)


    #outlier_dates1 = data1[data1['Anomaly_Label_IQR'] == 1].index
    # obtain y value of anomalies to plot
    #A_values = [data1.loc[k]['Target_var'] for k in outlier_dates1]
    #fig.add_trace(go.Scatter(x=outlier_dates1, y=A_values, mode = 'markers', 
    #                name = i+ "-" + "Anomaly IQR",
    #                marker=dict(color='yellow',size=10)))
    #plt.legend(loc="upper center")
    #fig.update_xaxes(rangeslider_visible=True,)
    #fig.show()
    
    #outlier_dates1 = data1[data1['Anomaly_4wra'] == 1].index
    # obtain y value of anomalies to plot
    #A_values = [data1.loc[k]['Target_var'] for k in outlier_dates1]
    #fig.add_trace(go.Scatter(x=outlier_dates1, y=A_values, mode = 'markers', 
    #                name = i+ "-" + "Anomaly 4wra",
    #                marker=dict(color='blue',size=10)))
    #plt.legend(loc="upper center")
    #fig.update_xaxes(rangeslider_visible=True,)
    #fig.show()
    
    #test_scores = emmv_scores(model, data[data.columns])
    precision_IQR = precision_score(data1['Anomaly_Label_IQR'], data1['anomaly_iforest'], average='binary')
    #precision_4wra = precision_score(data1['Anomaly_4wra'], data1['anomaly_iforest'], average='binary')
    recall_IQR = recall_score(data1['Anomaly_Label_IQR'], data1['anomaly_iforest'], average='binary')
    #recall_4wra = recall_score(data1['Anomaly_4wra'], data1['anomaly_iforest'], average='binary')
    f1_IQR = f1_score(data1['Anomaly_Label_IQR'], data1['anomaly_iforest'], average='binary')
    #f1_4wra = f1_score(data1['Anomaly_4wra'], data1['anomaly_iforest'], average='binary')
    confusion_IQR = confusion_matrix(data1['Anomaly_Label_IQR'], data1['anomaly_iforest'])
    #confusion_4wra = confusion_matrix(data1['Anomaly_4wra'], data1['anomaly_iforest'])
    tn_IQR, fp_IQR, fn_IQR, tp_IQR = confusion_matrix(data1['Anomaly_Label_IQR'], data1['anomaly_iforest'], labels=[0, 1]).ravel()
    #tn_4wra, fp_4wra, fn_4wra, tp_4wra = confusion_matrix(data1['Anomaly_4wra'], data1['anomaly_iforest'], labels=[0, 1]).ravel()

    k=i.replace(" ", "_")
    k=k.replace("-", "_")
    modelname = 'Isolationforest'+k+'_model'
    modelnam_type = 'Isolationforest'+k+'_model.pkl'
    model_path = './Isolationforest'+k+'_model.pkl'
    #print(modelname)
    #print(modelnam_type)
    #print(model_path)
    joblib.dump(model, modelnam_type)
    model = Model.register(workspace=workspace,
                        model_name= modelname,                # Name of the registered model in your workspace.
                        model_path= model_path,  # Local file to upload and register as a model.
                        model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                        model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                        #sample_input_dataset=input_dataset,
                        #sample_output_dataset=output_dataset,
                        resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                        description='Isolation forest model for'+i,
                        tags={'area': 'ASPR', 'type': 'Anomaly detection'})

    #print('Name:', model.name)
    #print('Version:', model.version)

    #data1['em'] = test_scores['em']
    #data1['mv'] = test_scores['mv']
    data1['Category'] = i
    data1['precision_IQR'] = precision_IQR
    #data1['precision_4wra'] = precision_4wra
    data1['recall_IQR'] = recall_IQR
    #data1['recall_4wra'] = recall_4wra
    data1['f1_IQR'] = f1_IQR
    #data1['f1_4wra'] = f1_4wra
    data1['tn_IQR'] = tn_IQR
    data1['fp_IQR'] = fp_IQR
    data1['fn_IQR'] = fn_IQR
    data1['tp_IQR'] = tp_IQR
    #data1['tn_4wra'] = tn_4wra
    #data1['fp_4wra'] = fp_4wra
    #data1['fn_4wra'] = fn_4wra
    #data1['tp_4wra'] = tp_4wra
    
    data2['Category'] = i
    data3['Category'] = i
    #features_perchange['Category'] = i
    data2['OrderN'] = data1.index
    data2 = data2.set_index('OrderN')
    data3['OrderN'] = data1.index
    data3 = data3.set_index('OrderN')
    #features_perchange = features_perchange.set_index('OrderN')
    anomaly_iforest_IQR = anomaly_iforest_IQR.append(data1)
    featureimportance_global = featureimportance_global.append(data2)
    featureimportance_local = featureimportance_local.append(data3)
    #Activity_percentage_change_dy = Activity_percentage_change_dy.append(features_perchange)


# Extract all the files to csv
local_path3 = 'Output/Isolationforest_featureimportance_local.csv'
featureimportance_local.to_csv(local_path3)

local_path4 = 'Output/Isolationforest_featureimportance_global.csv'
featureimportance_global.to_csv(local_path4)

local_path5 = 'Output/Isolationforest_results.csv'
anomaly_iforest_IQR.to_csv(local_path5)


# ### Using the 1 class SVM algorithm for anomaly detection, explaining the anomalies using percentage change of feature values from previous week ending

anomaly_1clssvm_IQR = pd.DataFrame()
featureimportance_global_1clssvm = pd.DataFrame()
featureimportance_local_1clssvm = pd.DataFrame()
#Activity_percentage_change_1clssvm = pd.DataFrame()

for i in listofcategory:
    #print(i)
    
    data=df3[df3['Category']== i]
    
    data = data.drop(['Category'],axis=1)
    data = data.groupby(['OrderN']).sum()
    
    model = OneClassSVM(kernel = 'rbf', gamma = 0.001, nu = 0.01)
    model.fit(data[data.columns])
    data1=data.copy()
    data1['scores']=model.decision_function(data[data.columns])
    data1['anomaly_1clssvm']=model.predict(data[data.columns])
    data1.loc[data1['anomaly_1clssvm'] == 1,'anomaly_1clssvm'] = 0
    data1.loc[data1['anomaly_1clssvm'] == -1,'anomaly_1clssvm'] = 1

    #fig = px.line(data1, x=data1.index, y="Target_var",  template = 'plotly_dark')

    # create list of outlier_dates
    #outlier_dates = data1[data1['anomaly_1clssvm'] == 1].index
    # obtain y value of anomalies to plot
    #y_values = [data1.loc[i]['Target_var'] for i in outlier_dates]
    #fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
    #                name = i + "_" + "Anomaly", 
    #                marker=dict(color='red',size=10)))

    #fig.show()
    
    data7=data1.copy()
    data7['change percentage'] = data7['Target_var'].pct_change()
    data7['change percentage'] = abs(data7['change percentage'])
    data7['change percentage'] = data7['change percentage'].replace(np.inf, max(data7['change percentage']))
    data7 = data7.fillna(0)
    avgprcntgechnge1 = data7['change percentage'].mean()
    #print(avgprcntgechnge1)
    n1=50
    data8 = data7.iloc[-n1:]
    count1 = (data8[data8['change percentage'] > avgprcntgechnge1]['change percentage'].count())
    contaminationfactor1 = count1/n1
    #print(count1)
    #print(contaminationfactor1)
    factor1 = max(0.1,contaminationfactor1)
    #print(factor1)
    high_thresh1=(1.0+factor1)
    low_thresh1=(1.0-factor1)
    
    # generate the labels using 4 week rolling average technique and IQR technique for evaluation
    #data1['4W rolling average'] = abs(data1.Target_var).rolling(4).mean()
    #data1['lag1'] = data1['Target_var'].shift(1)
    #data1 = data1.reset_index()
    #for p in range(len(data1)):
    #    if data1.loc[p,'4W rolling average'] != "nan":
    #        if (abs(data1.loc[p,'Target_var']) > (data1.loc[p,'4W rolling average']*high_thresh1)) or (abs(data1.loc[p,'Target_var']) < (data1.loc[p,'4W rolling average']*low_thresh1)):
    #            data1.loc[p,'Anomaly_4wra'] = 1
    #        else:
    #            data1.loc[p,'Anomaly_4wra'] = 0      
    #data1= data1.set_index('Week_Ending') 
    #data1= data1.fillna(0)

    def iqr_bounds(scores):
            q1 = scores.quantile((factor1/2))
            q3 = scores.quantile(1-(factor1/2))
            #iqr = q3 - q1
            lower_bound=(q1)
            upper_bound=(q3)
            #print("Lower bound:{} \nUpper bound:{} \nq1:{} \nq3:{}".format(lower_bound,upper_bound,q1,q3))
            return lower_bound,upper_bound

    lower_bound,upper_bound=iqr_bounds(data1['Target_var'])

    data1['Anomaly_Label_IQR']=0
    data1['Anomaly_Label_IQR']=(data1['Target_var'] < lower_bound) |(data1['Target_var'] > upper_bound)
    data1['Anomaly_Label_IQR']=data1['Anomaly_Label_IQR'].astype(int) 
    
    # SVM is not supported by tree explainer hence we cant get feature importance using SHAP here.
    
    
    # visually see if the anomalies identified seem reasonable using by feeding data to PCA reduced to 2 dimensions
    data_2=data1.copy()
    data_2=data_2.reset_index()
    outliers=data_2.loc[data_2['anomaly_1clssvm']== 1]
    outlier_index=list(outliers.index)
    
    #pca = PCA(2)
    #pca.fit(data1[data.columns])
    #res=pd.DataFrame(pca.transform(data1[data.columns]))
    #Z = np.array(res)
    #plt.title("One Class SVM")
    #plt.contourf( Z, cmap=plt.cm.Blues_r)
    #b1 = plt.scatter(res[0], res[1], c='green',
    #                 s=20,label="normal points")
    #b1 =plt.scatter(res.iloc[outlier_index,0],res.iloc[outlier_index,1], c='green',s=20,  edgecolor="red",label="predicted outliers")
    #plt.legend(loc="upper right")
    #plt.show()
    
    # Identify anomalies for individual metrics and plot the results.
    # X axis — date
    # Y axis — Actual values and anomaly points.
    # Actual values of metrics are indicated in the blue line and anomaly points are highlighted as red points.
    # In the table, the background red indicates high anomalies and yellow indicates low anomalies.
    
    #features_perchange=pd.DataFrame()
    #for m in range(1,len(data_2.columns)-1):
    #    model.fit(data_2.iloc[:,m:m+1])
    #    pred = model.predict(data_2.iloc[:,m:m+1])
    #    test_df=pd.DataFrame()
    #    test_df['Week_Ending']=data_2['Week_Ending']
    #    #Find decision function to find the score and classify anomalies
    #    test_df['score']=model.decision_function(data_2.iloc[:,m:m+1])
    #    test_df['actuals']=data_2.iloc[:,m:m+1]
    #    test_df['anomaly']=pred
    #    #Get the indexes of outliers in order to compare the metrics with use case anomalies if required
    #    outliers=test_df.loc[test_df['anomaly']==-1]
    #    outlier_index=list(outliers.index)
    #    test_df=classify_anomalies(test_df,data_2.columns[m])
    #    features_perchange = features_perchange.append(test_df)


    #outlier_dates1 = data1[data1['Anomaly_Label_IQR'] == 1].index
    # obtain y value of anomalies to plot
    #A_values = [data1.loc[k]['Target_var'] for k in outlier_dates1]
    #fig.add_trace(go.Scatter(x=outlier_dates1, y=A_values, mode = 'markers', 
    #                name = i+ "-" + "Anomaly IQR", 
    #                marker=dict(color='yellow',size=20)))
    #fig.update_xaxes(rangeslider_visible=True,)
    #fig.show()
    
    precision_IQR = precision_score(data1['Anomaly_Label_IQR'], data1['anomaly_1clssvm'], average='binary')
    #precision_4wra = precision_score(data1['Anomaly_4wra'], data1['anomaly_1clssvm'], average='binary')
    recall_IQR = recall_score(data1['Anomaly_Label_IQR'], data1['anomaly_1clssvm'], average='binary')
    #recall_4wra = recall_score(data1['Anomaly_4wra'], data1['anomaly_1clssvm'], average='binary')
    f1_IQR = f1_score(data1['Anomaly_Label_IQR'], data1['anomaly_1clssvm'], average='binary')
    #f1_4wra = f1_score(data1['Anomaly_4wra'], data1['anomaly_1clssvm'], average='binary')
    tn_IQR, fp_IQR, fn_IQR, tp_IQR = confusion_matrix(data1['Anomaly_Label_IQR'], data1['anomaly_1clssvm'], labels=[0, 1]).ravel()
    #tn_4wra, fp_4wra, fn_4wra, tp_4wra = confusion_matrix(data1['Anomaly_4wra'], data1['anomaly_1clssvm'], labels=[0, 1]).ravel()

    k=i.replace(" ", "_")
    k=k.replace("-", "_")
    modelname = '1classvm'+k+'_model'
    modelnam_type = '1classvm'+k+'_model.pkl'
    model_path = './1classvm'+k+'_model.pkl'
    #print(modelname)
    #print(modelnam_type)
    #print(model_path)
    joblib.dump(model, modelnam_type)
    model = Model.register(workspace=workspace,
                        model_name= modelname,                # Name of the registered model in your workspace.
                        model_path= model_path,  # Local file to upload and register as a model.
                        model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                        model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                        #sample_input_dataset=input_dataset,
                        #sample_output_dataset=output_dataset,
                        resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                        description='One class SVM model for'+i,
                        tags={'area': 'ASPR', 'type': 'Anomaly detection'})

    #print('Name:', model.name)
    #print('Version:', model.version)

    data1['Category'] = i
    data1['precision_IQR'] = precision_IQR
    #data1['precision_4wra'] = precision_4wra
    data1['recall_IQR'] = recall_IQR
    #data1['recall_4wra'] = recall_4wra
    data1['f1_IQR'] = f1_IQR
    #data1['f1_4wra'] = f1_4wra
    data1['tn_IQR'] = tn_IQR
    data1['fp_IQR'] = fp_IQR
    data1['fn_IQR'] = fn_IQR
    data1['tp_IQR'] = tp_IQR
    #data1['tn_4wra'] = tn_4wra
    #data1['fp_4wra'] = fp_4wra
    #data1['fn_4wra'] = fn_4wra
    #data1['tp_4wra'] = tp_4wra
    #data1['Category'] = i
    #features_perchange['Category'] = i
    #features_perchange = features_perchange.set_index('Week_Ending')
    anomaly_1clssvm_IQR = anomaly_1clssvm_IQR.append(data1)
    #Activity_percentage_change_1clssvm = Activity_percentage_change_1clssvm.append(features_perchange)

# Extract all the files to csv

local_path6 = 'Output/1clssvm_results.csv'
anomaly_1clssvm_IQR.to_csv(local_path6)

#local_path7 = 'Output/Activity_percentage_change_1clssvm.csv'
#Activity_percentage_change_1clssvm.to_csv(local_path7)


anomaly_1clssvm_IQR.drop(['Anomaly_Label_IQR','f1_IQR','precision_IQR','recall_IQR','tn_IQR','fp_IQR','fn_IQR','tp_IQR'], inplace=True, axis=1)
#anomaly_1clssvm_IQR = anomaly_1clssvm_IQR.rename(columns = {'SIO_Growth_x':'SIO_Growth'}, inplace = False)

# #### RUN random forest classifier using arima labels

anomaly_RF_IQR = pd.DataFrame()
oneclssvm_featureimportance_global = pd.DataFrame()
oneclssvm_featureimportance_local = pd.DataFrame()
#1clssvm_Activity_percentage_change = pd.DataFrame()

for i in listofcategory:
    
    data=anomaly_1clssvm_IQR[anomaly_1clssvm_IQR['Category']== i]
    
    data = data.drop(['Category'],axis=1)
    data = data.groupby(['OrderN']).sum()
    
    data4 = data.copy()
    data4['change percentage'] = data4['Target_var'].pct_change()
    data4['change percentage'] = abs(data4['change percentage'])
    data4['change percentage'] = data4['change percentage'].replace(np.inf, max(data4['change percentage']))
    data4 = data4.fillna(0)
    avgprcntgechnge = data4['change percentage'].mean()
    n=50
    data6 = data4.iloc[-n:]

    count = (data6[data6['change percentage'] > avgprcntgechnge]['change percentage'].count())
    contaminationfactor = count/n

    factor = max(0.1,contaminationfactor)
    
    high_thresh=(1.0+factor)
    low_thresh=(1.0-factor)

    # split into X and y
    X = data4.drop(['anomaly_1clssvm','change percentage'], axis=1)
    y = data4['anomaly_1clssvm']
    X = X.reset_index()
    y = y.reset_index()
    X = X.groupby(['OrderN']).sum()
    y = y.groupby(['OrderN']).sum()
    
    data7=X.copy()
    data7['change percentage'] = data7['Target_var'].pct_change()
    data7['change percentage'] = abs(data7['change percentage'])
    data7 = data7.fillna(0)
    avgprcntgechnge1 = data7['change percentage'].mean()
    n1=50
    data8 = data7.iloc[-n1:]
    count1 = (data8[data8['change percentage'] > avgprcntgechnge1]['change percentage'].count())
    contaminationfactor1 = count1/n1
    factor1 = max(0.1,contaminationfactor1)
    high_thresh1=(1.0+factor1)
    low_thresh1=(1.0-factor1)
    
    X = X.drop(['Target_var'], axis=1)
    RF_model = RandomForestRegressor()
    RF = RF_model.fit(X,y)
    data1=X.copy()
    data1['anomaly_RF'] = RF.predict(X)
    data1['anomaly_1classvm']=y['anomaly_1clssvm']

    # get the global feature importances at segment-product level using SHAP
    exp = shap.TreeExplainer(RF) #Explainer
    data = data.drop(['Target_var'], axis=1)
    shap_values = exp.shap_values(data)  #Calculate SHAP values
    #shap.summary_plot(shap_values, data, plot_type="bar", title=i+ "-" + "Global importance")
    data2 = pd.DataFrame()
    for j in range(0, len(data)):
        data2=data2.append(dict(zip(data.columns, shap_values[j])), ignore_index=True)

    # get the local feature importances at each index level for every segment-product combination using SHAP
    #data = data.drop(['Target_var'], axis=1)
    explainer = shap.TreeExplainer(RF, data=data[data.columns])
    shapvs = explainer.shap_values(data[data.columns], check_additivity=True) # check should be default

    data3=pd.DataFrame()
    for l in range(0,len(data)):
        data3 = data3.append(dict(zip(data.columns, shapvs[l])), ignore_index=True) 
        
    data_2=data1.copy()
    data_2['Target_var'] = data7['Target_var']
    data_2=data_2.reset_index()
    #features_perchange=pd.DataFrame()
    #for m in range(1,len(data_2.columns)-1):
    #    test_df=pd.DataFrame()
    #    test_df['Week_Ending']=data_2['Week_Ending']
    #    test_df['actuals']=data_2.iloc[:,m:m+1]
    #    test_df=classify_anomalies(test_df,data_2.columns[m])
    #    features_perchange = features_perchange.append(test_df)
    
    for n in data1.index:
        if (data1.loc[n,'anomaly_RF'] > 0.1):
            data1.loc[n,'anomaly_RF'] = 1
        else:
            data1.loc[n,'anomaly_RF'] = 0
    
    precision_IQR = precision_score(data1['anomaly_1classvm'], data1['anomaly_RF'])
    recall_IQR = recall_score(data1['anomaly_1classvm'], data1['anomaly_RF'])
    f1_IQR = f1_score(data1['anomaly_1classvm'], data1['anomaly_RF'])
    confusion_IQR = confusion_matrix(data1['anomaly_1classvm'], data1['anomaly_RF'])
    tn_IQR, fp_IQR, fn_IQR, tp_IQR = confusion_matrix(data1['anomaly_1classvm'], data1['anomaly_RF'], labels=[0, 1]).ravel()
    
    data1['Target_var'] = data7['Target_var']    
    
    data1['Category'] = i
    data1['precision_IQR'] = precision_IQR
    data1['recall_IQR'] = recall_IQR
    data1['f1_IQR'] = f1_IQR
    data1['tn_IQR'] = tn_IQR
    data1['fp_IQR'] = fp_IQR
    data1['fn_IQR'] = fn_IQR
    data1['tp_IQR'] = tp_IQR
    data3['Category'] = i
    data3['OrderN'] = data1.index
    data3 = data3.set_index('OrderN')
    data2['Category'] = i
    data2['OrderN'] = data1.index
    data2 = data2.set_index('OrderN')
    oneclssvm_featureimportance_global = oneclssvm_featureimportance_global.append(data2)
    anomaly_RF_IQR = anomaly_RF_IQR.append(data1)
    oneclssvm_featureimportance_local = oneclssvm_featureimportance_local.append(data3)
    #1clssvm_Activity_percentage_change = 1clssvm_Activity_percentage_change.append(features_perchange)


# Extract all the files to csv

local_path7 = 'Output/oneclssvm_featureimportance_local.csv'
oneclssvm_featureimportance_local.to_csv(local_path7)

local_path8 = 'Output/oneclssvm_featureimportance_global.csv'
oneclssvm_featureimportance_global.to_csv(local_path8)

#local_path3 = 'Output/Arima_Activity_percentage_change.csv'
#Arima_Activity_percentage_change.to_csv(local_path3)


all_results = pd.DataFrame()
all_results = anomaly_iforest_IQR[['Category']]
all_results['IForest_f1_IQR'] = anomaly_iforest_IQR['f1_IQR']
all_results['SVM_f1_IQR'] = anomaly_1clssvm_IQR['f1_IQR']
all_results = all_results.reset_index()
#all_results['RNN_f1_IQR'] = anomaly_RNN_IQR['f1_IQR']
#all_results['Arima_f1_IQR'] = arima_results['f1_IQR']

all_results = all_results.groupby(['Category']).mean()

#var_results = var_results.groupby(['Segment-Product']).mean()
#all_results['Var_f1_IQR']=var_results['f1_IQR']

all_results['Best_model'] = all_results.idxmax(axis=1)

best_model_across_segments = all_results['Best_model'].mode()

local_path9 = 'Output/all_results.csv'
all_results.to_csv(local_path9)

# get the datastore to upload prepared data
datastore = workspace.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir = 'Output', target_path = 'ASPR_Anomaly_Detection' ,overwrite=True)



Overwriting ASPR_pipeline/aspr_script.py


In [6]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DockerConfiguration
#from azureml.core import ScriptRunConfig

#Create a Python environment for the experiment (from a .yml file)

experiment_env = Environment.from_conda_specification("python_env2", experiment_folder + "/python_env2.yml")


experiment_env.register(workspace=ws)

# registered_env = Environment.get(ws, 'python_env2')
registered_env = Environment.get(ws,name="python_env2",version="18")

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

No Python version provided, defaulting to "3.8.12"


In [7]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep



# Run the  train script
train_step = PythonScriptStep(name = "train model",
                                source_directory = experiment_folder ,
                                script_name = "aspr_script.py",
                                #compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = False)

In [8]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails


# Construct the pipeline
pipeline_steps = [train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps,description="Traning")
print("Pipeline is built.")


# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'ASPR_experiment')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
#RunDetails(pipeline_run).show()

pipeline_run.wait_for_completion(show_output=True)



Pipeline is built.
Created step train model [f1e42c26][58bb0a63-aead-4c64-bb8c-69194feb6238], (This step will run and generate new outputs)
Submitted PipelineRun 023980d1-4262-40d4-9a72-ab4572c51559
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/023980d1-4262-40d4-9a72-ab4572c51559?wsid=/subscriptions/c4f0bb45-07a9-489a-8f6a-6fbc64f25b46/resourcegroups/tcp-azu0044-ae-rg-dcml-uc001-dt/workspaces/tcp0044aedcmluc001dtmlws&tid=49dfc6a3-5fb7-49f4-adea-c54e725bb854
Pipeline submitted for execution.
PipelineRunId: 023980d1-4262-40d4-9a72-ab4572c51559
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/023980d1-4262-40d4-9a72-ab4572c51559?wsid=/subscriptions/c4f0bb45-07a9-489a-8f6a-6fbc64f25b46/resourcegroups/tcp-azu0044-ae-rg-dcml-uc001-dt/workspaces/tcp0044aedcmluc001dtmlws&tid=49dfc6a3-5fb7-49f4-adea-c54e725bb854
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 98fbc6bd-6c99-4413-81e3-20a86901658d
Link to Azure Machine Learning Portal: h

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process '/azureml-envs/azureml_2985c9e26d721291a108505866e36889/bin/python' exited with code 1 and error message 'Execution failed. Process exited with status code 1. Error: Traceback (most recent call last):\\n  File \\\"aspr_script.py\\\", line 875, in <module>\\n    all_results['SVM_f1_IQR'] = anomaly_1clssvm_IQR['f1_IQR']\\n  File \\\"/azureml-envs/azureml_2985c9e26d721291a108505866e36889/lib/python3.6/site-packages/pandas/core/frame.py\\\", line 2906, in __getitem__\\n    indexer = self.columns.get_loc(key)\\n  File \\\"/azureml-envs/azureml_2985c9e26d721291a108505866e36889/lib/python3.6/site-packages/pandas/core/indexes/base.py\\\", line 2900, in get_loc\\n    raise KeyError(key) from err\\nKeyError: 'f1_IQR'\\n\\n'. Please check the log file 'user_logs/std_log.txt' for more details.\"}\n{\n  \"code\": \"ExecutionFailed